## Automating Disaster Mapping Processes
### Cases from flood disasters in the cities of the Global South

This code will turn raster data into an interactive web map dispalying the extent of flooding events and its effects on the local population and infrastructure.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import osmnx as ox
import shapely
from shapely.geometry import Point, LineString, Polygon, shape
from shapely.geometry import box
import rasterio as rio
import rasterio.features
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.features import shapes
from rasterio.mask import mask
from fiona.crs import from_epsg
import pycrs
import os
import mapclassify

### Raster transformations

In [2]:
# Filepath for WSF raster file
wsf_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/WSF/Bangkok_WSF2019population_10m.tif'

wsf_raster = rio.open(wsf_fp)

In [3]:
# Filepath for ICEYE raster file
iceye_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/ICEYE/Bangkok_iceye.tif'

iceye_orig = rio.open(iceye_fp)

In [4]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Getting CRS from the WSF data
dstCrs = wsf_raster.crs

# Calculate transform array and shape of reprojected layer
transform, width, height = calculate_default_transform(iceye_orig.crs, dstCrs, iceye_orig.width, iceye_orig.height, *iceye_orig.bounds)

# Working of the meta for the destination raster
kwargs = iceye_orig.meta.copy()
kwargs.update({'crs': dstCrs, 'transform': transform, 'width': width, 'height': height})

iceye_raster = rio.open(iceye_fp, 'w', **kwargs)

# Reproject and save raster band data
for i in range(1, iceye_raster.count +1):
    reproject(
        source=rio.band(iceye_orig, i),
        destination=rio.band(iceye_raster, i),
        src_crs=iceye_raster.crs,
        dstCrs=dstCrs,
        resampling=Resampling.nearest)

# Close destination raster    
iceye_raster.close()

print('Progress: Raster reprojection done.')

# Reopening the raster that is now projected to EPSG: 4326
iceye_wgs84 = rio.open(iceye_fp)

Progress: Raster reprojection done.


### Vector transformations

In [5]:
# Polygonizing the raster file
mask = None
with rio.Env():
    with rio.open(iceye_fp) as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

# Creating a GeoDataFrame from the polygonized raster
iceye_poly = gpd.GeoDataFrame.from_features(geoms)

# Get indexes where 'raster_val' column has value 'min'
indexNames = iceye_poly[iceye_poly['raster_val'] == iceye_poly['raster_val'].min()].index
# Delete 'min' rows indexes from dataframe as they are Null values in the raster file
iceye_poly.drop(indexNames, inplace=True)

print('Progress: Polygonizing done.')

Progress: Polygonizing done.


In [6]:
iceye_breaks = iceye_poly

# Rounding up unnecessary decimals
iceye_breaks['raster_val'] = iceye_breaks['raster_val'].round(3)

# Removing all rows with the value of zero
iceye_breaks = iceye_breaks[iceye_breaks.raster_val != 0.000]

# Classifying the data by natural breaks into 6 classes
breaks = mapclassify.NaturalBreaks.make(k=6)
iceye_breaks['naturalBreaks'] = iceye_breaks[['raster_val']].apply(breaks)

# Dissolving the data into the 6 classified breaks
iceye_breaks = iceye_breaks.dissolve(by='naturalBreaks', as_index=False)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [7]:
iceye_breaks

,naturalBreaks,geometry,raster_val
0,0,"MULTIPOLYGON (((100.33636 13.67740, 100.33636 ...",0.128
1,1,"MULTIPOLYGON (((100.33911 13.67273, 100.33911 ...",1.083
2,2,"MULTIPOLYGON (((100.33966 13.65623, 100.33993 ...",1.118
3,3,"MULTIPOLYGON (((100.33993 13.65623, 100.34021 ...",1.762
4,4,"MULTIPOLYGON (((100.44416 13.53907, 100.44416 ...",6.357
5,5,"MULTIPOLYGON (((100.40759 13.71260, 100.40731 ...",9.344


## Raster caclulations

### Raster clipping and affected population estimation

In [8]:
results = []

for i in iceye_breaks['naturalBreaks']:
        
    roi = iceye_breaks[iceye_breaks.naturalBreaks == i]
        
    gtraster, bound = rio.mask.mask(wsf_raster, roi['geometry'], crop=True)
        
    results.append(gtraster[0][gtraster[0]>0].sum())
    
iceye_breaks['population'] = results

# Dividing by 1000 because the WSF data has been multiplied by 1000 to save the file as integer
iceye_breaks['population'] = iceye_breaks['population'].div(1000).round(0)

In [9]:
iceye_breaks

,naturalBreaks,geometry,raster_val,population
0,0,"MULTIPOLYGON (((100.33636 13.67740, 100.33636 ...",0.128,814239.0
1,1,"MULTIPOLYGON (((100.33911 13.67273, 100.33911 ...",1.083,386215.0
2,2,"MULTIPOLYGON (((100.33966 13.65623, 100.33993 ...",1.118,171850.0
3,3,"MULTIPOLYGON (((100.33993 13.65623, 100.34021 ...",1.762,61700.0
4,4,"MULTIPOLYGON (((100.44416 13.53907, 100.44416 ...",6.357,5235.0
5,5,"MULTIPOLYGON (((100.40759 13.71260, 100.40731 ...",9.344,905.0


In [10]:
# Creating a constant value to use for dissolve
iceye_breaks['dissolve'] = 1

# Dissolving all the flooded polygons into one for extent of flooding
iceye_dissolve = iceye_breaks.dissolve(by='dissolve')

In [11]:
iceye_dissolve

,geometry,naturalBreaks,raster_val,population
dissolve,,,,
1,"MULTIPOLYGON (((100.34048 13.65650, 100.34048 ...",0,0.128,814239.0


#### Getting OSM data from flooded area

In [12]:
# Creating a convex hull of the flooded area
convex_hull = iceye_dissolve.convex_hull

# Turning the GeoSeries into a GeoDataFrame
flood_aoi = gpd.GeoDataFrame(geometry=gpd.GeoSeries(convex_hull))

# Turning the GeoDataFrame into a shapely polygon for OSMnx
osm_aoi = flood_aoi.iloc[0]['geometry']

# Getting OpenStreetMap data from the flooded area
#hospital = ox.geometries.geometries_from_polygon(osm_aoi, tags={'amenity':'hospital'})
#pharmacy = ox.geometries.geometries_from_polygon(osm_aoi, tags={'amenity':'pharmacy'})
buildings = ox.geometries.geometries_from_polygon(osm_aoi, tags={'building': True})

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/geometries.py:816: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_inner_linestring in list(merged_inner_linestrings):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/geometries.py:816: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part ge

In [13]:
buildings

created_by  \
element_type osmid                  
node         280179417        NaN   
             891268834        NaN   
             891270814        NaN   
             891270838        NaN   
             891270839        NaN   
...                           ...   
relation     13906434         NaN   
             13907161         NaN   
             13909935         NaN   
             13919460         NaN   
             13919616         NaN   

                                                                 geometry  \
element_type osmid                                                          
node         280179417                         POINT (100.55983 13.72984)   
             891268834                         POINT (100.56967 13.83939)   
             891270814                         POINT (100.57014 13.84995)   
             891270838                         POINT (100.57009 13.85141)   
             891270839                         POINT (100.57118 13.85152)   
...                                                                   ...   
relation     13906434   POLYGON ((100.72766 13.85624, 100.72798 13.856...   
             13907161   POLYGON ((100.70895 13.89786, 100.70900 13.897...   
             13909935   POLYGON ((100.72022 13.88190, 100.72033 13.881...   
             13919460   POLYGON ((100.54065 13.76007, 100.54056 13.759...   
             13919616   POLYGON ((100.53075 13.75911, 100.53081 13.759...   

                       barrier                name name:en    building  \
element_type osmid                                                       
node         280179417     NaN                 NaN     NaN         yes   
             891268834     NaN       บ้านชูวิเชียร     NaN  apartments   
             891270814     NaN      คณะสังคมศาสตร์     NaN  university   
             891270838     NaN      คณะศึกษาศาสตร์     NaN  university   
             891270839     NaN  คณะอุตสาหกรรมเกษตร     NaN  university   
...                        ...                 ...     ...         ...   
relation     13906434      NaN                 NaN     NaN         yes   
             13907161      NaN                 NaN     NaN         yes   
             13909935      NaN                 NaN     NaN         yes   
             13919460      NaN        Picnic Hotel     NaN         yes   
             13919616      NaN                 NaN     NaN         yes   

                                                      note layer entrance  \
element_type osmid                                                          
node         280179417  MRT ventilation and emergency exit   NaN      NaN   
             891268834                                 NaN   NaN      NaN   
             891270814                                 NaN   NaN      NaN   
             891270838                                 NaN   NaN      NaN   
             891270839                                 NaN   NaN      NaN   
...                                                    ...   ...      ...   
relation     13906434                                  NaN   NaN      NaN   
             13907161                                  NaN   NaN      NaN   
             13909935                                  NaN   NaN      NaN   
             13919460                                  NaN   NaN      NaN   
             13919616                                  NaN   NaN      NaN   

                       highway  ... company massage                      ways  \
element_type osmid              ...                                             
node         280179417     NaN  ...     NaN     NaN                       NaN   
             891268834     NaN  ...     NaN     NaN                       NaN   
             891270814     NaN  ...     NaN     NaN                       NaN   
             891270838     NaN  ...     NaN     NaN                       NaN   
             891270839     NaN  ...     NaN     NaN                       NaN   
...    

In [ ]:
build_test = buildings[['geometry', 'building']]

In [ ]:
# Making a copy of the points as this iterration drops points as they are assigned to polygons to speed up the process
pts = build_test.copy()

# Creating an empty list to store the amount of points
in_flood = [0]

# Loop over polygons with index i
for i, poly in iceye_breaks.iterrows():
    
    # List of points in polygon
    in_this_poly = []
    
    # Looping over all points with index j
    for j, pt in pts.iterrows():
        if poly.geometry.contains(poly.geometry):
            # There is a point in this polygon, add it to the list. Point is dropped to have less points to go through next time around the loop.
            in_this_poly.append(poly.geometry)
            pts = pts.drop([j])
            
    # Appending the number of points into the list
    in_flood.append(len(in_this_poly))

In [ ]:
print(in_flood)

In [ ]:
m = folium.Map(location=[13.7, 100.6],
              zoom_start=11,
              control_scale=True,
              tiles='CartoDB Positron')

In [ ]:
folium.GeoJson(buildings).add_to(m)

## Raster plotting

### ICEYE flood data plotting

In [ ]:
iceye_breaks.plot(column='naturalBreaks', scheme='quantiles', cmap='Blues')

### FOLIUM

In [ ]:
m = folium.Map(location=[13.7, 100.6],
              zoom_start=11,
              control_scale=True,
              tiles='CartoDB Positron')

In [ ]:
# Setting a CRS to the data
test_dissolve.crs = 'epsg:4326'

# Creating a Geo-id that Folium needs for plotting, it needs to have a unique indetifier for each row (Tenkanen & al. 2022)
test_dissolve['geoid'] = test_dissolve.index.astype(str)

folium.Choropleth(geo_data=test_dissolve,
                 name='Flood map',
                 data=test_dissolve,
                 columns=['geoid', 'naturalBreaks'],
                 key_on='feature.id',
                 fill_color='Blues',
                 fill_opacity=0.7,
                 line_opacity=0.2,
                 line_color='white',
                 line_weight=0,
                 highlight=False,
                 smooth_factor=1.0,
                 legend_name='Flood in Bangkok').add_to(m)

In [ ]:
m

### MAPBOX

In [ ]:
test_gjson = test.to_json()

In [ ]:
token = os.getenv('pk.eyJ1Ijoib2h0b255Z3JlbiIsImEiOiJja2tyMmowZ2YwZTU4MndvNm4yMW84OXhrIn0.lwnlCLxsvFIPqn7yzyGxXw')

In [ ]:
from mapboxgl.viz import *
from mapboxgl.utils import *
import mapclassify

In [ ]:
color_breaks = mapclassify.NaturalBreaks(test['raster_val'], k=8).bins
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

viz = ChoroplethViz('test_gjson', access_token=token, 
                    height='400px', 
                    color_property = 'raster_val', 
                    color_stops = color_stops, 
                    center = (13.7, 100.6), 
                    zoom=11)

### LEAFMAP

In [ ]:
import os
import leafmap.leafmap as leafmap

In [ ]:
#out_dir = os.path.expanduser('~/Yliopisto/Gradu/Data/ICEYE/')

#if not os.path.exists(out_dir):
    #os.makedirs(out_dir)

#flood_raster = os.path.join(out_dir, 'Bangkok_iceye.tif')

In [ ]:
out_dir = os.path.expanduser('~/Yliopisto/Gradu/Data/ICEYE/')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

flood_raster = os.path.join(out_dir, 'Bangkok_iceye.tif')

In [ ]:
Map = leafmap.Map()

In [ ]:
Map.add_raster(flood_raster, colormap='coolwarm', layer_name='Flood')

In [ ]:
Map.to_html('Thesis_map.html')